<a href="https://colab.research.google.com/github/luizrenault/tp547/blob/main/lista_filas/questao3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install matplotlib

**Lista de Filas**

**3)** Um comutador de pacotes recebe em média 200 pacotes/segundo, cada um com
um comprimento médio de 128 bytes. O comutador possui uma única linha de
saída com capacidade de 256 kbps. Considere um buffer com {1,5,10,15}
posições na fila, qual a probabilidade de bloqueio, número médio de elementos e
tempo médio no sistema?

$ \lambda = 200 $ pacotes/segundo


$ E[t_s] = \frac{1}{\mu} = 128 * 8 / 256000 = 1/250 = 0,004 $ segundo

$ \rho = \frac{\lambda}{\mu} = 200/250 = 0,8 $

$ N = J + 1 =$ {2, 6, 11, 16}

$ P_b = \rho^N \frac{1-\rho}{1-\rho^{N+1}} = $\{0,26229508 0,06634165 0,01844758 0,00575918\}

$ E[q] = \frac{\rho}{1 - \rho} - \frac{(N+1)\rho^{N+1}}{1 - \rho^{N+1}} = $ \{0,85245902 2,14243372 3,1145163  3,60837543\} elementos

$ E[t_q] = \frac{E[q]}{(1-P_b)\lambda} = $ \{0,00577778 0,01147333 0,01586526 0,01814639\} segundo

**Solução por simulação**

O código abaixo foi executado para J = {1,5,10,15}, obtendo os seguintes resultados:

$ P_b = $\{0,2689126 0,0678582 0,015886959 0,00576563\}

$ E[q] = $ \{1,1729965 2,2905936 3,082494 3,5578339\} elementos

$ E[t_q] = $ \{0,00586498 0,011452968 0,01541247 0,0177891696\} segundo


In [21]:
import numpy as np
import matplotlib.pyplot as plt

# Parâmetros

lambda1=200 #chegadas/tempo - taxa media de chegada
mu1=250 #partidas/tempo - taxa media de partida
Nq=15 #tamanho da fila

#cálculos auxiliares para solução analítica
rho = lambda1 / mu1
J = np.array([1, 5, 10, 15])
N = J + 1
rhoN = rho ** N
rhoN1 = rho ** (N + 1)
Pb = rhoN * (1-rho) / (1-rhoN1)

print(f'Prob Bloqueio Analítica: {Pb}')

Eq = rho / (1-rho) - (N+1) * rhoN1 / (1-rhoN1)

print(f'Elementos no sistema (analítica): {Eq}')

Etq = Eq / ((1-Pb) * lambda1)

print(f'Tempo no sistema (analítica): {Etq}')

t = 0 #tempo inicial
lq = 0 #numero de usuários na fila
ls = 1 #numero de usuários no servidor
k1 = 1 #número de chegadas
k2 = 1 #número de serviços
k = 0 #numero de partidas
b = 0 #numero de bloqueios

TS=200 #tempo máximo de simulação

def chegada():
  return np.random.exponential(1/lambda1)

def partida():
  return np.random.exponential(1/mu1)

tc = [0] #tempo de chegadas
tp = [] #tempo de partidas
ts = [] #tempo no sistema

ta = t + chegada() #proxima chegada
td = t + partida() #proxima partida
t = np.minimum(ta, td) #proximo tempo de execucao

while (t < TS): #tempo limite de simulacao

    if ta<td: #evento de chegada
        k1 = k1 + 1 #incrementa o número de chegadas
        if lq < Nq:
            lq = lq + 1 #aumenta a fila
            tc = np.append(tc, t) #aramazena tempo atual de chegada
            #print(f"CF({lq})S({ls})T({t})")
        else:
            b = b + 1 #incrementa o número de bloqueios
            #print(f"BF({lq})S({ls})T({t})")
        ta = t + chegada() #gera nova chegada

    else: #evento de partida
        k = k + 1 #incrementa o numero de partidas
        tp = np.append(tp, t) #armazeno o tempo da partida
        ts = np.append(ts, (tp[k - 1] - tc[k - 1])) #calculo o tempo no sistema
        td = np.infty #coloco um valor muito grande para a proxima partida
        ls = 0 #esvazio o servidor
        #print(f"PF({lq})S({ls})T({t})")

    if ls == 0 and lq > 0: #servidor desocupado e pacote na fila
        lq = lq - 1
        td = t + partida() #gera nova partida
        k2 = k2 + 1
        ls = 1 #ocupa o servidor
        #print(f"SF({lq})S({ls})T({t})")

    t = np.minimum(ta, td) #calculo o tempo minimo entre chegada e partida

tw=np.mean(ts)-1/mu1
eq=lambda1*np.mean(ts)
ew=lambda1*tw
rho=lambda1/mu1
pb = b/k1

print(f"fator de utilizacao rho: {rho}")
print(f"tempo no sistema: {np.mean(ts)}")
print(f"tempo na fila: {tw}")
print(f"numero medio de usuários no sistema: {eq}")
print(f"numero medio de usuários na fila: {ew}")
print(f"probabilidade de bloqueios: {pb}")
print(f"bloqueios: {b}")
print(f"chegadas: {k1}")
print(f"fila: {lq}")
print(f"entrada sv: {k2}")
print(f"partidas: {k}")

Prob Bloqueio Analítica: [0.26229508 0.06634165 0.01844758 0.00575918]
Elementos no sistema (analítica): [0.85245902 2.14243372 3.1145163  3.60837543]
Tempo no sistema (analítica): [0.00577778 0.01147333 0.01586526 0.01814639]
fator de utilizacao rho: 0.8
tempo no sistema: 0.017789169655368264
tempo na fila: 0.013789169655368264
numero medio de usuários no sistema: 3.557833931073653
numero medio de usuários na fila: 2.7578339310736526
probabilidade de bloqueios: 0.005765630849868963
bloqueios: 231
chegadas: 40065
fila: 3
entrada sv: 39831
partidas: 39830
